In [21]:
import pandas as pd

# File paths
first_100_games_file = 'first_100_games.csv'
filtered_merged_events_file = 'filtered_merged_events.csv'

# Step 1: Read the CSV files into DataFrames
print(f"Reading '{first_100_games_file}'...")
first_100_games_data = pd.read_csv(first_100_games_file)
print(f"'{first_100_games_file}' loaded with {len(first_100_games_data)} rows.")

print(f"Reading '{filtered_merged_events_file}'...")
filtered_merged_events_data = pd.read_csv(filtered_merged_events_file)
print(f"'{filtered_merged_events_file}' loaded with {len(filtered_merged_events_data)} rows.")

# Step 2: Concatenate both DataFrames
print("Concatenating the DataFrames...")
combined_data = pd.concat([first_100_games_data, filtered_merged_events_data], ignore_index=True)
print(f"Combined DataFrame has {len(combined_data)} rows.")

# Step 3: Sort the combined DataFrame by fixture_id
print("Sorting the combined DataFrame by 'fixture_id'...")
combined_data = combined_data.sort_values(by='fixture_id').reset_index(drop=True)
print("DataFrame sorted.")

# Step 4: Display the first few rows of the combined DataFrame
print("Displaying the first 5 rows of the sorted DataFrame:")
print(combined_data.head())

# Step 5: Optionally save to a new CSV
output_file = 'combined_sorted_by_fixture_id.csv'
print(f"Saving the combined DataFrame to '{output_file}'...")
combined_data.to_csv(output_file, index=False)
print(f"Data saved as '{output_file}'.")


Reading 'first_100_games.csv'...
'first_100_games.csv' loaded with 16749 rows.
Reading 'filtered_merged_events.csv'...
'filtered_merged_events.csv' loaded with 114772 rows.
Concatenating the DataFrames...
Combined DataFrame has 131521 rows.
Sorting the combined DataFrame by 'fixture_id'...
DataFrame sorted.
Displaying the first 5 rows of the sorted DataFrame:
   fixture_id event_time  team_id event_type      detailed_type  \
0         113         91       45       Pass    Pass_Successful   
1         113         48       45       Pass    Pass_Successful   
2         113         36       48       Pass  Pass_Unsuccessful   
3         113         36       48       Pass    Pass_Successful   
4         113         31       48       Pass    Pass_Successful   

   main_player_id  secondary_player_id  
0          2990.0                  NaN  
1         18760.0                  NaN  
2          2412.0                  NaN  
3          2412.0                  NaN  
4         18826.0             

In [24]:
# Number of rows in the DataFrame
num_rows = combined_data.shape[0]

# List of column names
column_names = combined_data.columns.tolist()

# Unique values in the 'event_type' column
unique_event_types = combined_data['event_type'].unique()

# Print results
print(f"Number of rows in the DataFrame: {num_rows}")
print(f"List of column names: {column_names}")
print(f"Unique values in 'event_type' column: {unique_event_types}")


Number of rows in the DataFrame: 131521
List of column names: ['fixture_id', 'event_time', 'team_id', 'event_type', 'detailed_type', 'main_player_id', 'secondary_player_id']
Unique values in 'event_type' column: ['Pass' 'BallRecovery' 'Foul' 'BallTouch' 'Interception' 'Clearance'
 'Aerial' 'Challenge' 'Dispossessed' 'TakeOn' 'KeeperPickup' 'Goal' 'Card'
 'subst' 'SubstitutionOff' 'OffsideGiven' 'MissedShots' 'ShotOnPost'
 'KeeperSweeper' 'Possession' 'FormationChange' 'FormationSet' 'Error'
 'SavedShot' 'BlockedPass' 'Save' 'CornerAwarded' 'GoodSkill'
 'ShieldBallOpp' 'Tackle' 'OffsidePass' 'OffsideProvoked' 'SubstitutionOn'
 'Punch' 'Claim' 'PenaltyFaced' 'Smother' 'ChanceMissed' 'CrossNotClaimed'
 'Var']


In [25]:
# make all the data in column event_time to be string
# Convert all values in the "event_time" column to strings
combined_data['event_time'] = combined_data['event_time'].astype(str)

In [26]:
# Count the number of missing values in each column
missing_values = combined_data.isnull().sum()
# Filter and print only the columns with missing values greater than zero
non_zero_missing = missing_values[missing_values > 0]
print(non_zero_missing)
# check how much rows in the db
# num_rows = data.count()
# print(num_rows)

main_player_id          11322
secondary_player_id    128167
dtype: int64


In [27]:
# check that all the main player id missing is from types which not include players
# Create a set to store `detailed_type` values where `main_player_id` is empty
detailed_type_set = set()

# Iterate through the rows where `main_player_id` is NaN or empty
for idx, row in combined_data.iterrows():
    if pd.isna(row['main_player_id']) or row['main_player_id'] == '':
        detailed_type_set.add(row['event_type'])

# Display the resulting set
print(detailed_type_set)

# Filter rows where `main_player_id` is missing
missing_main_player = combined_data[combined_data['main_player_id'].isna() | (combined_data['main_player_id'] == '')]

# Count the occurrences of each `detailed_type` for missing `main_player_id`
missing_counts = missing_main_player['event_type'].value_counts()

# Print the results
for event_type, count in missing_counts.items():
    print(f"Missing values for event_type '{event_type}': {count}")


# after check randomly - the data is missing from the api 
# FormationSet,Possession,FormationChange is ok that we have missing values - it's not related to specific player
# card missing values we will drop
# subst and goal - it is importat to know that the event happe, but not who did it (at least for now) we will fill the main player as generic player with id 0



{'Possession', 'FormationSet', 'FormationChange'}
Missing values for event_type 'Possession': 9772
Missing values for event_type 'FormationChange': 1106
Missing values for event_type 'FormationSet': 444


In [28]:
# Create a copy of the original data to work on
updated_data = combined_data.copy()

# Filter rows where `main_player_id` is missing
missing_main_player = updated_data[updated_data['main_player_id'].isna() | (updated_data['main_player_id'] == '')]

# Count the missing occurrences for each `event_type`
missing_counts = missing_main_player['event_type'].value_counts()
print("Missing value counts by `event_type`:")
print(missing_counts)

# Process the missing values based on your logic
for idx, row in updated_data.iterrows():
    if pd.isna(row['main_player_id']) or row['main_player_id'] == '':
        event_type = row['event_type']

        if event_type in {"FormationSet", "Possession", "FormationChange"}:
            # It's okay to have missing values, so do nothing
            continue

        elif event_type == "Card":
            # Drop rows with `event_type` as "card"
            updated_data.drop(index=idx, inplace=True)

        elif event_type in {"subst", "Goal"}:
            # Replace missing `main_player_id` with a generic player ID (0)
            updated_data.at[idx, 'main_player_id'] = 0

# Reset the index of the updated DataFrame after dropping rows
updated_data.reset_index(drop=True, inplace=True)

# Verify the results
print("Processing complete. Updated DataFrame:")
print(updated_data.head())

# Filter rows where `main_player_id` is missing
missing_main_player = updated_data[updated_data['main_player_id'].isna() | (updated_data['main_player_id'] == '')]

# Count the missing occurrences for each `event_type`
missing_counts = missing_main_player['event_type'].value_counts()
print("Missing main player value counts by `event_type`:")
print(missing_counts)


Missing value counts by `event_type`:
event_type
Possession         9772
FormationChange    1106
FormationSet        444
Name: count, dtype: int64
Processing complete. Updated DataFrame:
   fixture_id event_time  team_id event_type      detailed_type  \
0         113         91       45       Pass    Pass_Successful   
1         113         48       45       Pass    Pass_Successful   
2         113         36       48       Pass  Pass_Unsuccessful   
3         113         36       48       Pass    Pass_Successful   
4         113         31       48       Pass    Pass_Successful   

   main_player_id  secondary_player_id  
0          2990.0                  NaN  
1         18760.0                  NaN  
2          2412.0                  NaN  
3          2412.0                  NaN  
4         18826.0                  NaN  
Missing main player value counts by `event_type`:
event_type
Possession         9772
FormationChange    1106
FormationSet        444
Name: count, dtype: int64


In [29]:
# Function to process the `event_time` column
def process_event_time(value):
    try:
        if value.isdigit():
            # If the value is a number as a string, convert it to an integer
            return int(value)
        # Check if the value is a negative number as a string
        elif value.lstrip('-').isdigit():
            return int(value)
        elif '+' in value:
            base, extra = value.split('+')
            if base == "90":
                # For "90+something", return the integer sum
                return int(base) + int(extra)
            elif base == "45":
                # For "45+something", return a float "45.something"
                return float(f"{base}.{extra}")
        else:
            raise ValueError(f"Unexpected format: {value}")
    except Exception as e:
        print(f"Error processing value '{value}': {e}")
        return None  # Return None for invalid cases

# Apply the function to the `event_time` column in the updated_data DataFrame
updated_data['event_time'] = updated_data['event_time'].apply(process_event_time)

# Verify the results
print("Processed DataFrame (updated_data):")
print(updated_data.head())


# Count the number of non-integer values in the event_time column
non_float_count = updated_data['event_time'].apply(lambda x: not isinstance(x, float)).sum()

print(f"Number of non-float values in the 'event_time' column: {non_float_count}")



Processed DataFrame (updated_data):
   fixture_id  event_time  team_id event_type      detailed_type  \
0         113        91.0       45       Pass    Pass_Successful   
1         113        48.0       45       Pass    Pass_Successful   
2         113        36.0       48       Pass  Pass_Unsuccessful   
3         113        36.0       48       Pass    Pass_Successful   
4         113        31.0       48       Pass    Pass_Successful   

   main_player_id  secondary_player_id  
0          2990.0                  NaN  
1         18760.0                  NaN  
2          2412.0                  NaN  
3          2412.0                  NaN  
4         18826.0                  NaN  
Number of non-float values in the 'event_time' column: 0


In [30]:
# Create a dictionary to store counts of event types with `event_time < 0`
negative_event_time_dict = {}

# Iterate over rows where `event_time` is less than 0
for idx, row in updated_data.iterrows():
    if row['event_time'] < 0:
        event_type = row['event_type']
        # Increment the count for the event type in the dictionary
        if event_type in negative_event_time_dict:
            negative_event_time_dict[event_type] += 1
        else:
            negative_event_time_dict[event_type] = 1

# Print the dictionary
print(negative_event_time_dict)
print("Counts of rows with `event_time < 0` by `event_type`:")
for event_type, count in negative_event_time_dict.items():
    print(f"{event_type}: {count}")


# we will drop the rows if the type is card else we will check based on the number of problematic values
# Count the number of rows that match the condition
rows_to_delete = updated_data[(updated_data['event_time'] < 0) & (updated_data['event_type'] == "Card")].shape[0]

# Drop rows where `event_time` < 0 and `event_type` is "Card"
updated_data = updated_data[~((updated_data['event_time'] < 0) & (updated_data['event_type'] == "Card"))]

# Reset the index of the DataFrame after dropping rows
updated_data.reset_index(drop=True, inplace=True)

# Print the number of rows deleted
print(f"Number of rows deleted: {rows_to_delete}")

# check if it indeed delete
# Create a dictionary to store counts of event types with `event_time < 0`
negative_event_time_dict = {}

# Iterate over rows where `event_time` is less than 0
for idx, row in updated_data.iterrows():
    if row['event_time'] < 0:
        event_type = row['event_type']
        # Increment the count for the event type in the dictionary
        if event_type in negative_event_time_dict:
            negative_event_time_dict[event_type] += 1
        else:
            negative_event_time_dict[event_type] = 1

# Print the dictionary
print(negative_event_time_dict)
print("Counts of rows with `event_time < 0` by `event_type`:")
for event_type, count in negative_event_time_dict.items():
    print(f"{event_type}: {count}")

{'Card': 1}
Counts of rows with `event_time < 0` by `event_type`:
Card: 1
Number of rows deleted: 1
{}
Counts of rows with `event_time < 0` by `event_type`:


In [31]:
# every missing secondary player we will fill with the "default" player - 0
# Fill NaN values in the 'secondary_player' column with 0
updated_data['secondary_player_id'] = updated_data['secondary_player_id'].fillna(0)

# Verify the changes
print("Updated DataFrame with 'secondary_player_id' NaN values filled:")
print(updated_data.head())

Updated DataFrame with 'secondary_player_id' NaN values filled:
   fixture_id  event_time  team_id event_type      detailed_type  \
0         113        91.0       45       Pass    Pass_Successful   
1         113        48.0       45       Pass    Pass_Successful   
2         113        36.0       48       Pass  Pass_Unsuccessful   
3         113        36.0       48       Pass    Pass_Successful   
4         113        31.0       48       Pass    Pass_Successful   

   main_player_id  secondary_player_id  
0          2990.0                  0.0  
1         18760.0                  0.0  
2          2412.0                  0.0  
3          2412.0                  0.0  
4         18826.0                  0.0  


In [32]:
# Step 1: Sort the DataFrame by fixture_id and event_time
print("Sorting the DataFrame by 'fixture_id' and 'event_time'")
updated_data = updated_data.sort_values(by=['fixture_id', 'event_time']).reset_index(drop=True)
print("DataFrame sorted.")

# Step 2: Display the first 5 rows of the sorted DataFrame
print("Displaying the first 5 rows of the sorted DataFrame:")
print(updated_data.head())



Sorting the DataFrame by 'fixture_id' and 'event_time'
DataFrame sorted.
Displaying the first 5 rows of the sorted DataFrame:
   fixture_id  event_time  team_id    event_type    detailed_type  \
0         113         0.0       48  FormationSet          4-1-4-1   
1         113         0.0       48  FormationSet          4-1-4-1   
2         113         0.0       45          Pass  Pass_Successful   
3         113         0.0       45          Pass  Pass_Successful   
4         113         0.0       48  FormationSet          4-1-4-1   

   main_player_id  secondary_player_id  
0             NaN                  0.0  
1             NaN                  0.0  
2          2990.0                  0.0  
3         18764.0                  0.0  
4             NaN                  0.0  


In [33]:
# Step 1: Count the number of duplicate rows
duplicate_count = updated_data.duplicated().sum()
print(f"Number of duplicate rows in the DataFrame: {duplicate_count}")

Number of duplicate rows in the DataFrame: 31739


In [34]:
# Step 3: Remove duplicate rows
print(f"Number of rows before removing duplicates: {len(updated_data)}")
updated_data = updated_data.drop_duplicates().reset_index(drop=True)
print(f"Number of rows after removing duplicates: {len(updated_data)}")


Number of rows before removing duplicates: 131520
Number of rows after removing duplicates: 99781


In [35]:
# Step 1: Create a new column for possession_detailed_type
updated_data['possession_detailed_type'] = updated_data.apply(
    lambda row: row['detailed_type'] if row['event_type'] == 'Possession' else 0, axis=1
)

# Step 2: Update the `detailed_type` column to 0 where `event_type` is `Possession`
updated_data['detailed_type'] = updated_data.apply(
    lambda row: 0 if row['event_type'] == 'Possession' else row['detailed_type'], axis=1
)

# Display the updated DataFrame
print(updated_data.head())

   fixture_id  event_time  team_id    event_type         detailed_type  \
0         113         0.0       48  FormationSet               4-1-4-1   
1         113         0.0       45          Pass       Pass_Successful   
2         113         0.0       45          Pass       Pass_Successful   
3         113         0.0       45        Aerial   Aerial_Unsuccessful   
4         113         0.0       48     BallTouch  BallTouch_Successful   

   main_player_id  secondary_player_id possession_detailed_type  
0             NaN                  0.0                        0  
1          2990.0                  0.0                        0  
2         18764.0                  0.0                        0  
3         18766.0                  0.0                        0  
4         18818.0                  0.0                        0  


In [36]:
# do one hot encoding to the event type columns
# Step 1: Perform One-Hot Encoding on the `event_type` column
event_type_encoded = pd.get_dummies(updated_data['event_type'], prefix='event_type')
# Step 2: Merge the one-hot encoded columns back to the original DataFrame
updated_data = pd.concat([updated_data, event_type_encoded], axis=1)
updated_data.drop(columns=['event_type'], inplace=True)
# convert in into int and not true/false
dummy_cols = [col for col in updated_data.columns if any(prefix in col for prefix in ['event_type_'])]
updated_data[dummy_cols] = updated_data[dummy_cols].astype(int)

# Display the updated DataFrame
print(updated_data.head())

   fixture_id  event_time  team_id         detailed_type  main_player_id  \
0         113         0.0       48               4-1-4-1             NaN   
1         113         0.0       45       Pass_Successful          2990.0   
2         113         0.0       45       Pass_Successful         18764.0   
3         113         0.0       45   Aerial_Unsuccessful         18766.0   
4         113         0.0       48  BallTouch_Successful         18818.0   

   secondary_player_id possession_detailed_type  event_type_Aerial  \
0                  0.0                        0                  0   
1                  0.0                        0                  0   
2                  0.0                        0                  0   
3                  0.0                        0                  1   
4                  0.0                        0                  0   

   event_type_BallRecovery  event_type_BallTouch  ...  event_type_SavedShot  \
0                        0                 

In [37]:
# do one hot encoding to the event type columns
# Step 1: Perform One-Hot Encoding on the `event_type` column
detailed_type_encoded = pd.get_dummies(updated_data['detailed_type'], prefix='detailed_type')
# Step 2: Merge the one-hot encoded columns back to the original DataFrame
updated_data = pd.concat([updated_data, detailed_type_encoded], axis=1)
updated_data.drop(columns=['detailed_type'], inplace=True)
# convert in into int and not true/false
dummy_cols = [col for col in updated_data.columns if any(prefix in col for prefix in ['detailed_type_'])]
updated_data[dummy_cols] = updated_data[dummy_cols].astype(int)

# Display the updated DataFrame
print(updated_data.head())

   fixture_id  event_time  team_id  main_player_id  secondary_player_id  \
0         113         0.0       48             NaN                  0.0   
1         113         0.0       45          2990.0                  0.0   
2         113         0.0       45         18764.0                  0.0   
3         113         0.0       45         18766.0                  0.0   
4         113         0.0       48         18818.0                  0.0   

  possession_detailed_type  event_type_Aerial  event_type_BallRecovery  \
0                        0                  0                        0   
1                        0                  0                        0   
2                        0                  0                        0   
3                        0                  1                        0   
4                        0                  0                        0   

   event_type_BallTouch  event_type_BlockedPass  ...  \
0                     0                       0 

In [38]:
# Identify columns with non-numeric values
non_numeric_columns = []

for col in updated_data.columns:
    if not pd.api.types.is_numeric_dtype(updated_data[col]):
        non_numeric_columns.append(col)

# Print the columns with non-numeric values
print("Columns with non-numeric values:")
for col in non_numeric_columns:
    print(col)


Columns with non-numeric values:
possession_detailed_type


In [39]:
# Identify and print non-numeric values in the `possession_detailed_type` column
non_numeric_values = updated_data['possession_detailed_type'][~updated_data['possession_detailed_type'].apply(lambda x: isinstance(x, (int, float)))]

print("Non-numeric values in `possession_detailed_type` column:")
print(non_numeric_values)


Non-numeric values in `possession_detailed_type` column:
140      57.1
141      42.9
253      56.0
254      44.0
396      56.5
         ... 
99653    69.7
99767    28.0
99769    72.0
99779    72.0
99780    28.0
Name: possession_detailed_type, Length: 2202, dtype: object


In [40]:
# Function to convert values to float
def to_float(value):
    try:
        # Attempt to convert the value to float
        return float(value)
    except (ValueError, TypeError):
        # Handle non-numeric values gracefully
        return None  # Or you can set a default value like 0.0

# Apply the function to the `possession_detailed_type` column
updated_data['possession_detailed_type'] = updated_data['possession_detailed_type'].apply(to_float)

# Print the updated column to verify
print("Updated `possession_detailed_type` column:")
print(updated_data['possession_detailed_type'])


Updated `possession_detailed_type` column:
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
99776     0.0
99777     0.0
99778     0.0
99779    72.0
99780    28.0
Name: possession_detailed_type, Length: 99781, dtype: float64


In [41]:
print(updated_data.columns.to_list())

['fixture_id', 'event_time', 'team_id', 'main_player_id', 'secondary_player_id', 'possession_detailed_type', 'event_type_Aerial', 'event_type_BallRecovery', 'event_type_BallTouch', 'event_type_BlockedPass', 'event_type_Card', 'event_type_Challenge', 'event_type_ChanceMissed', 'event_type_Claim', 'event_type_Clearance', 'event_type_CornerAwarded', 'event_type_CrossNotClaimed', 'event_type_Dispossessed', 'event_type_Error', 'event_type_FormationChange', 'event_type_FormationSet', 'event_type_Foul', 'event_type_Goal', 'event_type_GoodSkill', 'event_type_Interception', 'event_type_KeeperPickup', 'event_type_KeeperSweeper', 'event_type_MissedShots', 'event_type_OffsideGiven', 'event_type_OffsidePass', 'event_type_OffsideProvoked', 'event_type_Pass', 'event_type_PenaltyFaced', 'event_type_Possession', 'event_type_Punch', 'event_type_Save', 'event_type_SavedShot', 'event_type_ShieldBallOpp', 'event_type_ShotOnPost', 'event_type_Smother', 'event_type_SubstitutionOff', 'event_type_SubstitutionO

In [43]:
# Columns to drop and check for values
columns_to_drop = [
    'event_type_BallTouch', 'event_type_Error', 'event_type_GoodSkill',
    'event_type_Punch', 'event_type_Save', 'event_type_SubstitutionOff', 'event_type_SubstitutionOn'
]

# Step 1: Remove rows where any of the specified columns have a value of 1
print(f"Number of rows before filtering: {len(updated_data)}")
updated_data = updated_data[~updated_data[columns_to_drop].eq(1).any(axis=1)]
print(f"Number of rows after removing rows with value 1 in specified columns: {len(updated_data)}")

# Step 2: Drop the specified columns
updated_data = updated_data.drop(columns=columns_to_drop)
print(f"Dropped columns: {columns_to_drop}")


Number of rows before filtering: 99781
Number of rows after removing rows with value 1 in specified columns: 94549
Dropped columns: ['event_type_BallTouch', 'event_type_Error', 'event_type_GoodSkill', 'event_type_Punch', 'event_type_Save', 'event_type_SubstitutionOff', 'event_type_SubstitutionOn']


In [44]:
# Columns to drop and check for values
columns_to_drop = [
    'detailed_type_BallTouch_Successful', 'detailed_type_BallTouch_Unsuccessful',
    'detailed_type_Error_Successful', 'detailed_type_GoodSkill_Successful',
    'detailed_type_Save_Successful', 'detailed_type_SubstitutionOff_Successful',
    'detailed_type_SubstitutionOn_Successful'
]

# Step 1: Remove rows where any of the specified columns have a value of 1
print(f"Number of rows before filtering: {len(updated_data)}")
updated_data = updated_data[~updated_data[columns_to_drop].eq(1).any(axis=1)]
print(f"Number of rows after removing rows with value 1 in specified columns: {len(updated_data)}")

# Step 2: Drop the specified columns
updated_data = updated_data.drop(columns=columns_to_drop)
print(f"Dropped columns: {columns_to_drop}")

# Count the number of columns
num_columns = len(updated_data.columns)
print(f"Number of columns in the DataFrame: {num_columns}")


Number of rows before filtering: 94549
Number of rows after removing rows with value 1 in specified columns: 94549
Dropped columns: ['detailed_type_BallTouch_Successful', 'detailed_type_BallTouch_Unsuccessful', 'detailed_type_Error_Successful', 'detailed_type_GoodSkill_Successful', 'detailed_type_Save_Successful', 'detailed_type_SubstitutionOff_Successful', 'detailed_type_SubstitutionOn_Successful']
Number of columns in the DataFrame: 111


In [45]:
# Save the cleaned DataFrame
output_file = '100_games_events_table.csv'
updated_data.to_csv(output_file, index=False)
print(f"Cleaned DataFrame saved as '{output_file}'.")

Cleaned DataFrame saved as '100_games_events_table.csv'.
